# Tugas PCA
## Timothy Dillan Tandjung (NIM: 20917054)

# Dataset
Dataset yang digunakan adalah Movie Conversations yang disediakan di [kaggle](https://www.kaggle.com/cashutosh/gender-classification-dataset).

Dataset yang digunakan pada metode ini berasal dari Kaggle, diunggah pada tahun 2019, berisi 23.766 ribu gambar laki-laki, dan 23.243 ribu gambar perempuan. Dataset gambar yang digunakan juga mengandung berbagai macam ekspresi dan ras.

# Importing Libraries
Untuk memulai workflow machine learning pada dataset, beberapa library seperti pandas, numpy, os, sklearn, dan openCV (cv2) akan digunakan untuk melakukan pra pemrosesan pada data serta membuat model untuk mengenali jenis kelamin.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Prapemrosesan
## Gray Scaling, Resize 80x80
Dalam proses prapemrosesan, terdapat beberapa tahapan yang dijalankan, yakni gray scaling, resize 80x80, standardization, dan reduksi dimensi. Pada tahap gray scaling, setiap gambar dijadikan hitam putih agar setiap gambar memiliki warna yang terstandardidasi dan mempercepat proses komputasi. Setelah itu, setiap gambar dibentuk ulang ke dalam bentuk 80x80 pixelsnya agar setiap gambar memiliki dimensi yang seragam.
Keseragaman dimensi penting dalam tahap ini karena dalam proses pelatihan sebuah model, dibutuhkan dimensi yang seragam untuk keseluruhan input.

In [2]:
genders = []
images = []
train_path = 'gender-images'
list_of_genders = os.listdir(train_path)

# Untuk setiap gender yang ada di dalam dataset,
for gender in list_of_genders:
    try:
        # Ambil nama setiap gambar
        pictures = os.listdir(train_path+'/'+gender)
    except:
        continue
    # Dan untuk setiap gambar
    for person_img in pictures:
        # Load gambarnya
        image = cv2.imread(train_path + "/" + gender + "/" + person_img)    
        # Lakukan *gray-scaling* pada gambar
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Resize ukuran menjadi 80x80
        image = cv2.resize(image, (80, 80))
        # Lalu tambahkan data gambar ke list gambar
        images.append(image)
        # dan tambahkan corresponding gendernya ke list gender
        genders.append(gender)

In [3]:
len(images)

47009

In [4]:
# Merubah list gambar menjadi bentuk matriks n * 80x80 = 47.009 * 6400
# 47.009 merujuk ke jumlah gambar, dan 6400 merujuk ke dimensi setiap gambar.
images_arr = np.array(images).reshape(len(images), images[0].shape[0]*images[0].shape[1])

# Standardization 
Setelah setiap gambar memiliki ukuran yang seragam, standardidasi atau penskalaan data perlu dilakukan agar setiap fitur yang digunakan dapat dibandingkan dengan bobot yang sama (Pedregosa et al., 2021). Penskalaan fitur dilakukan dengan cara mentransformasi setiap nilai pada dataset ke standard scorenya

In [5]:
# Split dataset gambar, dengan 70% untuk training dan 30% untuk testing.
train_img, test_img, train_lbl, test_lbl = train_test_split(images_arr, genders, test_size=0.3, random_state=1)

In [6]:
# Standardidasi dataset, dengan memfitting sebuah model standardidasi dengan data training,
# dan lalu mentransformasi data training dan testing.
scaler = StandardScaler().fit(train_img)
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

# PCA 
Lalu, pada tahap terakhir, PCA digunakan untuk mereduksi dimensi dari setiap gambar, dengan 248 principal components (fitur yang akan digunakan untuk pelatihan model) dan jumlah persentase variance yang dipertahankan sebesar 95%. Dengan direduksinya dimensi data menggunakan PCA, variabel redundant dapat dengan mudah dibuang dan model pembelajaran mesin dapat dengan mudah dan cepat dilatih.

In [7]:
# Membuat sebuah model PCA dengan data training yang memperthankan 95% persentase variance
pca = PCA(n_components=0.95).fit(train_img)

In [8]:
# Jumlah persentase variance model PCA
round(pca.explained_variance_ratio_.sum(), 5)

0.95004

In [9]:
# Transformasi data training dan testing dengan model PCA yang telah dibuat
pca_train = pca.transform(train_img)
pca_test = pca.transform(test_img)

In [10]:
# Mengecek shape hasil transformasi data training
# dapat dilihat bahwa shape data training sekarang berbentuk (32906, 125)
# 32906 = jumlah gambar, 125 = jumlah dimensi
pca_train.shape

(32906, 248)

In [11]:
# Mengecek shape hasil transformasi data training
# dapat dilihat bahwa shape data training sekarang berbentuk (14103, 125)
# 14103 = jumlah gambar, 125 = jumlah dimensi
pca_test.shape

(14103, 248)

# Pelatihan & Evaluasi Model
Setelah tahap prapemrosesan selesai, dataset yang telah diproses dibagi menjadi dua dataset untuk tujuan pelatihan dan evaluasi model. Untuk dataset pelatihan, 70% data dari dataset akan diambil, dan untuk evaluasi model, 30% sisa data dari dataset akan diambil. Lalu, library sklearn digunakan untuk membangun sebuah model SVM dengan parameter bawaan yang dilatih dengan dataset training tersebut. Model SVM yang telah dilatih lalu dievaluasi nilai precision, recall, F1-score, dan akurasinya pada dataset yang telah dikesampingkan.

In [12]:
# Melatih model klasifikasi SVM dengan data training yang telah ditransformasi
svc_clf = SVC().fit(pca_train, train_lbl)

In [13]:
# Mengevaluasi performa model yang telah dilatih dengan mencoba memprediksi data testing
# lalu dibandingkan dengan label sesungguhnya
svc_clf_report = classification_report(test_lbl, svc_clf.predict(pca_test))

In [14]:
# Hasil performa model
print("SVM Classifier")
print(svc_clf_report)

SVM Classifier
              precision    recall  f1-score   support

      female       0.92      0.91      0.91      6907
        male       0.91      0.93      0.92      7196

    accuracy                           0.92     14103
   macro avg       0.92      0.92      0.92     14103
weighted avg       0.92      0.92      0.92     14103



Dapat dilihat bahwa dari laporan evaluasi performa model diatas, model memiliki nilai F1-score sebesar 91% dan 92% untuk klasifikasi perempuan dan laki-laki, serta memiliki rata-rata nilai akurasi sebesar 92%.